In [1]:
#Robert Bickley - Implementation of Honors Thesis Analysis Concept

In [2]:
import pandas as pd
import numpy as np
import math as m

In [3]:
#Functionality:
#compare_boards will compare the state of two boards, one tuned and one un-tuned, and deliver a tuning recipe
#recipe_travel will determine the total amount of tuner travel that a recipe calls for
#recipe_wear will find the number of tuners that travel "significantly", based on a user-specified threshold
#recommend will assess a set of recipes and recommend one based on user-specified priorities

In [16]:
#Example Boards
#As in our previous analytical work for this project, data will be stored in pandas DataFrame objects
#An "un-tuned" board example, with only 10 tuners for this demonstration:
untuned_board = pd.DataFrame({'tuner':[0,1,2,3,4,5,6,7,8,9],'depth':[0.05,0.4,0.98,0.3,1.0,0.73,0.51,0.22,0.68,0.07]})
#Two possible "tuned" board examples, note that these do not correspond to a real tuned RF filter
tuned_board_1 = pd.DataFrame({'tuner':[0,1,2,3,4,5,6,7,8,9],'depth':[0.3,0.4,0.65,0.33,0.09,0.5,0.25,0.1,0.76,0.3]})
tuned_board_2 = pd.DataFrame({'tuner':[0,1,2,3,4,5,6,7,8,9],'depth':[0.1,0.67,0.11,0.45,0.14,1.02,0.54,0.31,0.74,0.03]})

In [17]:
def compare_boards(sample, tuned):
    #where "sample" is the board to be tuned, "tuned" is a known tuned board state
    #assumes boards are formatted the same way and have the same dimensions
    #returns a recipe, wherein positive values mean that a tuner should be driven in, negative means drawn out
    recipe = tuned['depth']-sample['depth']
    return recipe

In [18]:
#testing the compare_boards function
print(compare_boards(untuned_board,tuned_board_1))
print(compare_boards(untuned_board,tuned_board_2))

0    0.25
1    0.00
2   -0.33
3    0.03
4   -0.91
5   -0.23
6   -0.26
7   -0.12
8    0.08
9    0.23
Name: depth, dtype: float64
0    0.05
1    0.27
2   -0.87
3    0.15
4   -0.86
5    0.29
6    0.03
7    0.09
8    0.06
9   -0.04
Name: depth, dtype: float64


In [19]:
def recipe_travel(sample, tuned):
    #calculates the total travel in inches required to follow a recipe
    #a secondary metric of tuning time
    recipe = compare_boards(sample, tuned)
    travel = 0
    for i in range(len(recipe)):
        travel += abs(recipe[i])
    return travel

In [20]:
#testing the recipe_travel function
print(recipe_travel(untuned_board,tuned_board_1))
print(recipe_travel(untuned_board,tuned_board_2))

2.44
2.7099999999999995


In [23]:
def recipe_wear(sample, tuned, threshold):
    #calculates the number of tuners that travel significantly, i.e. recipe calls for n tuners to travel more than (threshold) inches
    #a metric of the amount of wear on the tuners and bushings
    recipe = compare_boards(sample, tuned)
    n = 0
    for i in range(len(recipe)):
        if abs(recipe[i]) >= threshold:
            n += 1
    return n

In [25]:
#testing the recipe_wear function
print(recipe_wear(untuned_board,tuned_board_1,0.1))
print(recipe_wear(untuned_board,tuned_board_2,0.1))

7
5


In [30]:
def recommend(sample, tuned_set, threshold, travel_weight, wear_weight):
    #computes recipes for each of the boards in tuned_set
    #considers the two metrics defined above, recipe_travel and recipe_wear, with weights travel_weight & wear_weight
    #recommends a tuning recipe
    #travel_weight and wear_weight can be any two numbers, but their values should scale linearly with the relative
    #importances of the two factors. For example:
    #"Speed is twice as important as tuner wear." => travel_weight = 2 and wear_weight = 1
    #"I care much more about wear on individual tuners." => travel_weight = 1 and wear_weight = 10
    #"These factors are equally important." => travel_weight = 1 and wear_weight = 1
    travels = []
    wears = []
    for i in range(len(tuned_set)):
        tuned = tuned_set[i]
        current_travel = recipe_travel(sample, tuned)
        current_wear = recipe_wear(sample, tuned, threshold)
        travels.append(current_travel)
        wears.append(current_wear)
    #here I normalize the size of the two factors so that with equal specified weighting they would be considered evenly.    
    scale = np.average(travels)/np.average(wears)
    wears = [i * scale for i in wears]
    scores = []
    for i in range(len(tuned_set)):
        scores.append(wears[i]*wear_weight+travels[i]*travel_weight)
    rec_index = scores.index(min(scores))
    return compare_boards(sample, tuned_set[rec_index])

In [33]:
#testing the recommend function
tuned_set = [tuned_board_1, tuned_board_2]
print(recommend(untuned_board, tuned_set, 0.1, 1, 1))
#here, we recommend tuning to tuned_board_2 because the wear condition is more significantly different than the travel condition
print(recommend(untuned_board, tuned_set, 0.1, 10, 1))
#considering travel weight much more heavily, we recommend tuning to tuned_board_1
print(recommend(untuned_board, tuned_set, 0.1, 1, 10))
#considering wear more heavily results in an easy decision for tuned_board_2

0    0.05
1    0.27
2   -0.87
3    0.15
4   -0.86
5    0.29
6    0.03
7    0.09
8    0.06
9   -0.04
Name: depth, dtype: float64
0    0.25
1    0.00
2   -0.33
3    0.03
4   -0.91
5   -0.23
6   -0.26
7   -0.12
8    0.08
9    0.23
Name: depth, dtype: float64
0    0.05
1    0.27
2   -0.87
3    0.15
4   -0.86
5    0.29
6    0.03
7    0.09
8    0.06
9   -0.04
Name: depth, dtype: float64
